In [1]:
!pip install git+https://github.com/Liadrinz/transformers-unilm
!pip install git+https://github.com/huggingface/transformers
!pip uninstall -y llmdet
!pip install --no-cache-dir git+https://github.com/dr-phil/LLMDetFix.git
!pip install scikit-learn transformers nltk lightgbm numpy dask[dataframe]
!pip install datasets

  Cloning https://github.com/Liadrinz/transformers-unilm to /tmp/pip-req-build-jf7n9dyq
  Running command git clone --filter=blob:none --quiet https://github.com/Liadrinz/transformers-unilm /tmp/pip-req-build-jf7n9dyq
  Resolved https://github.com/Liadrinz/transformers-unilm to commit 07320e9fe224f6256f7ca3a7e33557d9ce7870a4
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-61vuhp1_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-61vuhp1_
  Resolved https://github.com/huggingface/transformers to commit 79254c9b61ac2e1a6e0f698c38b1654159095e2e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Found existing installation: llmdet 1.1.0
Uninstalling llmdet-1.1.0:
  Successfully uninstalled llmdet-1.1.0
  Cloning https://github.com/dr-phil/LLMDetFix.git to /tmp/pip-req-bui

In [ ]:
# !pip install --no-cache-dir git+https://github.com/dr-phil/LLMDetFix.git
# !pip install git+https://github.com/dr-phil/LLMDetFix.git

In [ ]:
HF_KEY = "HUGGING_FACE_KEY_HERE"

# from google.colab import userdata
# userdata.set('HF_TOKEN', HF_KEY)

from huggingface_hub import login
login(HF_KEY)

In [3]:
import llmdet

llmdet.load_probability()

text = "The actress was honoured for her role in 'The Reader' at the annual ceremony, which was held at the Royal Albert Hall. The film, which is based on the novel by the same name by Philip Roth, tells the story of a New York Times reporter who returns to his hometown to cover the death of his brother-in-law. Winslet plays his wife, with whom he has been divided since the death of their son.\nIn the film, Winslet plays the mother of the grieving brother-in-law.\nThe actress also won a Golden Globe for her role in the film at the ceremony in November.\nWinslet was also nominated for an Oscar for her role in 'The Reader'.\nThe 63-year-old Winslet was seen accepting her awards at the ceremony, where she was joined by her husband, John Krasinski, who has been nominated for best supporting actor in the film.\nWinslet and Krasinski met while"

# Detect, `text` is a string or string list
result = llmdet.detect(text)
print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Computing checksums: 100%|##########| 1/1 [00:27<00:00, 27.49s/it]

100%|██████████| 1/1 [00:00<00:00, 27.43it/s]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
100%|██████████| 1/1 [00:00<00:00, 178.02it/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly r

[{'OPT': 0.5382045360581533, 'GPT-2': 0.4468143848737435, 'UniLM': 0.011892155704528217, 'T5': 0.002482670431074466, 'Bloom': 0.00025650921330735136, 'GPT-neo': 0.00025635420975859746, 'LLaMA': 6.148550300788111e-05, 'Human_write': 1.827207473487535e-05, 'BART': 1.3631931692021782e-05}]


In [ ]:
ai_gen = []
ai_gen.append("Zimbabwe faces persistent economic hardships, including rampant inflation, an unstable currency system, and fundamental institutional problems, though government initiatives have occasionally attempted to address these issues.")
ai_gen.append("Zimbabwe's economic climate remains volatile, marked by persistent hyperinflation, currency instability, and high unemployment, despite government efforts to implement reforms and attract foreign investment.")

human_gen = []
human_gen.append("The economic climate of Zimbabwe is challenging to describe, given that a large majority of it is deemed an 'informal economy'.")
human_gen.append("Zimbabwe's formal economy mostly focuses on mining, with many state-subsidized enterprises supporting the sector. Other industries include textiles and machining.")

# Detect, `text` is a string or string list
res = []
# print(llmdet.detect(ai_gen[0])[0]['Human_write'])
res.append(llmdet.detect(ai_gen[0]))
res.append(llmdet.detect(ai_gen[1]))
res.append(llmdet.detect(human_gen[0]))
res.append(llmdet.detect(human_gen[1]))

print(res)

100%|██████████| 1/1 [00:00<00:00, 162.80it/s]

[[{'LLaMA': 0.9527042686571954, 'GPT-2': 0.028283831560142636, 'Human_write': 0.007116975851113579, 'T5': 0.006514740321263047, 'OPT': 0.004268137418895303, 'UniLM': 0.0008061671009773561, 'GPT-neo': 0.0001923096296841437, 'BART': 7.463283244043474e-05, 'Bloom': 3.893662828781601e-05}], [{'LLaMA': 0.9663381504412846, 'OPT': 0.020678938300301406, 'GPT-2': 0.007964076802750871, 'Human_write': 0.0029483860727261696, 'T5': 0.0014540182803336758, 'UniLM': 0.00032550897110294395, 'GPT-neo': 0.00024562358690606085, 'Bloom': 4.237764661478816e-05, 'BART': 2.919897979572249e-06}], [{'Human_write': 0.8822308947693072, 'GPT-2': 0.04051440407345082, 'LLaMA': 0.03950710848732061, 'OPT': 0.019841599914779753, 'T5': 0.012435018542065688, 'UniLM': 0.0035587659703541896, 'GPT-neo': 0.0011851914012444632, 'Bloom': 0.0006175249104257927, 'BART': 0.00010949193105119382}], [{'Human_write': 0.7276283173038272, 'OPT': 0.09066788410044184, 'GPT-2': 0.09064445872803677, 'LLaMA': 0.07612214497951185, 'T5': 0.00

In [ ]:
for sublist in res:
    # Extract the dictionary from the sublist
    d = sublist[0]
    print(f"Human: {d['Human_write']}, AI: {1 - d['Human_write']}")
    max_key = max(d, key=d.get)
    print(f"Max key: {max_key}, Value: {d[max_key]}")

Human: 0.007116975851113579, AI: 0.9928830241488864
Max key: LLaMA, Value: 0.9527042686571954
Human: 0.0029483860727261696, AI: 0.9970516139272738
Max key: LLaMA, Value: 0.9663381504412846
Human: 0.8822308947693072, AI: 0.11776910523069284
Max key: Human_write, Value: 0.8822308947693072
Human: 0.7276283173038272, AI: 0.27237168269617285
Max key: Human_write, Value: 0.7276283173038272


In [ ]:
import os
from datasets import load_dataset, load_from_disk

SAVE_PATH = '/content/data/wiki_dataset'

def get_wikipedia_dataset():
    # Check if dataset already exists on disk
    if os.path.exists(SAVE_PATH):
        print("Loading dataset from disk...")
        dataset = load_from_disk(SAVE_PATH)
    else:
        print("Dataset not found on disk. Downloading from Hugging Face...")
        # Create directory if it doesn't exist
        os.makedirs('/content/data', exist_ok=True)

        # Download and load the dataset
        dataset = load_dataset("wikipedia", "20220301.en")

        # Save to disk
        print("Saving dataset to disk...")
        dataset.save_to_disk(SAVE_PATH)

    print("Dataset is ready!")
    return dataset

In [ ]:
legacy_dataset = get_wikipedia_dataset()

Dataset not found on disk. Downloading from Hugging Face...


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train-00000-of-00041.parquet:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [5]:
import json
from datasets import Dataset, DatasetDict

with open("/content/scraped_wiki_articles_8_2023.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

dataset_2023 = DatasetDict({"train": dataset})

print(dataset_2023)

DatasetDict({
    train: Dataset({
        features: ['title', 'url', 'text'],
        num_rows: 1000
    })
})


In [8]:
import json
import re

full_scores = []
all_scores = []
non_ai = 0
ai = 0
def ai_score(lines, truncate_to=5000):
    print(f"number of lines: {len(lines)}")
    global non_ai, ai
    scores = []
    for ind, line in enumerate(lines):

        print(len(line))
        full_score = llmdet.detect(' '.join(line.strip().split()[:truncate_to]))[0]
        full_scores.append(full_score)
        score = full_score['Human_write']
        print(f" combined: {full_score} \n {score}, {ind}")
        scores.append((score, ind))

        if max(full_score, key=full_score.get) != 'Human_write' and score < 4e-3:
            print("~~~ ai article ~~~")
            ai += 1
        else:
            non_ai += 1

    return scores

def clean_text(text):
    # Replace all newlines with a unique string
    text = text.replace('\n', '!@#').replace('\t', ' ')
    # Split the text by the unique string
    split_text = text.split('!@#')
    # Remove elements with 5 or fewer words
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    # Rejoin the text
    text = ' '.join(split_text)
    # Remove "References" and "External links" sections
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)

    return text.strip()

def extract_title_text_and_url_legacy():
    titles, texts, urls = [], [], []

    for i in range(1000):
    #   title = legacy_dataset["train"][i]["title"]
    #   content = legacy_dataset["train"][i]["text"]
    #   url = legacy_dataset["train"][i]["url"]
    #   cleaned_text = clean_text(content)

    #   for new
      title = dataset_2023["train"][i]["title"]
      content = dataset_2023["train"][i]["text"]
      url = dataset_2023["train"][i]["url"]
      cleaned_text = clean_text(content)

      if title and cleaned_text:
          titles.append(title)
          texts.append(cleaned_text)
          urls.append(url)

    return titles, texts, urls

def process_files(file_path, output_file):
    titles, texts, urls = extract_title_text_and_url_legacy()
    texts = [' '.join(text.strip().split()[:400]) for text in texts]
    final_texts, final_titles, final_urls = [],[],[]
    for ind, text in enumerate(texts):
        if len(text.split()) >= 100:
            final_texts.append(texts[ind])
            final_titles.append(titles[ind])
            final_urls.append(urls[ind])

    print(f"number of texts: {len(texts)}")

    bino_scores = ai_score(texts)
    all_scores.append(bino_scores)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for title, score, url, text in zip(final_titles, bino_scores, final_urls, final_texts):
            outfile.write(f"{title}, {score[0]}\n")


In [9]:
output_file = "/content/2023-wikipedia.json"
input_file = ""
process_files(input_file, output_file)

number of texts: 1000
number of lines: 1000
2345


100%|██████████| 1/1 [00:00<00:00, 14.60it/s]


 combined: {'Bloom': 0.7388888415695926, 'GPT-neo': 0.24398633650405055, 'Human_write': 0.00920576836625041, 'T5': 0.005932826074350915, 'BART': 0.0018467657250193345, 'LLaMA': 6.192582308623178e-05, 'GPT-2': 3.5556503218622706e-05, 'OPT': 2.8698325170204726e-05, 'UniLM': 1.3281109261249544e-05} 
 0.00920576836625041, 0
1275


100%|██████████| 1/1 [00:00<00:00, 24.34it/s]


 combined: {'T5': 0.8136035521015762, 'BART': 0.1318924717642639, 'GPT-neo': 0.038970803404042034, 'Human_write': 0.005864863195668, 'Bloom': 0.004437785657654677, 'UniLM': 0.003735366289500359, 'GPT-2': 0.0011469691481849407, 'LLaMA': 0.00019129224065509413, 'OPT': 0.00015689619845461058} 
 0.005864863195668, 1
949


...

 combined: {'LLaMA': 0.4004569898769987, 'Human_write': 0.2985531676607233, 'UniLM': 0.13937951130898496, 'T5': 0.05890511389287235, 'GPT-2': 0.05134610364047055, 'OPT': 0.04309680431064485, 'BART': 0.005279652182059959, 'GPT-neo': 0.0024602424102891985, 'Bloom': 0.0005224147169561145} 
 0.2985531676607233, 997
1598


100%|██████████| 1/1 [00:00<00:00, 19.05it/s]


 combined: {'GPT-neo': 0.7714511815761813, 'Bloom': 0.12890096296856304, 'T5': 0.09293535848837339, 'BART': 0.0053913106019915745, 'Human_write': 0.0004325843314595095, 'OPT': 0.00029485585889620964, 'LLaMA': 0.0002688780673643922, 'GPT-2': 0.00023347454543557884, 'UniLM': 9.1393561734918e-05} 
 0.0004325843314595095, 998
~~~ ai article ~~~
344


100%|██████████| 1/1 [00:00<00:00, 93.62it/s]

 combined: {'LLaMA': 0.4965185963824984, 'GPT-2': 0.2604897546608931, 'OPT': 0.12863355914239524, 'UniLM': 0.04626668443509765, 'Human_write': 0.036466562366228736, 'T5': 0.027961194393543862, 'GPT-neo': 0.001974108536375081, 'BART': 0.0014076009028782016, 'Bloom': 0.0002819391800896559} 
 0.036466562366228736, 999


In [14]:
import json
from datasets import Dataset, DatasetDict

with open("/content/scraped_wiki_articles_8_2024.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

dataset_2023 = DatasetDict({"train": dataset})

print(dataset_2023)

DatasetDict({
    train: Dataset({
        features: ['title', 'url', 'text'],
        num_rows: 850
    })
})


In [15]:
import json
import re

full_scores = []
all_scores = []
non_ai = 0
ai = 0
def ai_score(lines, truncate_to=5000):
    print(f"number of lines: {len(lines)}")
    global non_ai, ai
    scores = []
    for ind, line in enumerate(lines):

        print(len(line))
        full_score = llmdet.detect(' '.join(line.strip().split()[:truncate_to]))[0]
        full_scores.append(full_score)
        score = full_score['Human_write']
        print(f" combined: {full_score} \n {score}, {ind}")
        scores.append((score, ind))

        if max(full_score, key=full_score.get) != 'Human_write' and score < 4e-3:
            print("~~~ ai article ~~~")
            ai += 1
        else:
            non_ai += 1

    return scores

def clean_text(text):
    # Replace all newlines with a unique string
    text = text.replace('\n', '!@#').replace('\t', ' ')
    # Split the text by the unique string
    split_text = text.split('!@#')
    # Remove elements with 5 or fewer words
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    # Rejoin the text
    text = ' '.join(split_text)
    # Remove "References" and "External links" sections
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)

    return text.strip()

def extract_title_text_and_url_legacy():
    titles, texts, urls = [], [], []

    for i in range(850):
    #   title = legacy_dataset["train"][i]["title"]
    #   content = legacy_dataset["train"][i]["text"]
    #   url = legacy_dataset["train"][i]["url"]
    #   cleaned_text = clean_text(content)

    #   for new
      title = dataset_2023["train"][i]["title"]
      content = dataset_2023["train"][i]["text"]
      url = dataset_2023["train"][i]["url"]
      cleaned_text = clean_text(content)

      if title and cleaned_text:
          titles.append(title)
          texts.append(cleaned_text)
          urls.append(url)

    return titles, texts, urls

def process_files(file_path, output_file):
    titles, texts, urls = extract_title_text_and_url_legacy()
    texts = [' '.join(text.strip().split()[:400]) for text in texts]
    final_texts, final_titles, final_urls = [],[],[]
    for ind, text in enumerate(texts):
        if len(text.split()) >= 100:
            final_texts.append(texts[ind])
            final_titles.append(titles[ind])
            final_urls.append(urls[ind])

    print(f"number of texts: {len(texts)}")

    bino_scores = ai_score(texts)
    all_scores.append(bino_scores)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for title, score, url, text in zip(final_titles, bino_scores, final_urls, final_texts):
            outfile.write(f"{title}, {score[0]}\n")


In [16]:
output_file = "/content/2024-wikipedia.json"
input_file = ""
process_files(input_file, output_file)

number of texts: 850
number of lines: 850
2408


100%|██████████| 1/1 [00:00<00:00, 11.98it/s]


 combined: {'Bloom': 0.6316301897657404, 'GPT-neo': 0.25980358032826134, 'T5': 0.062245601378589456, 'BART': 0.034581367778963396, 'Human_write': 0.0112740060355383, 'LLaMA': 0.00019406717641580332, 'GPT-2': 9.763478225227731e-05, 'UniLM': 9.491535478734763e-05, 'OPT': 7.863739945178748e-05} 
 0.0112740060355383, 0
2377


100%|██████████| 1/1 [00:00<00:00, 15.28it/s]


 combined: {'Bloom': 0.5499772815451555, 'GPT-neo': 0.4368125976684351, 'T5': 0.00839880479456934, 'Human_write': 0.00400403288724895, 'BART': 0.0005388499500510845, 'LLaMA': 0.00010435647704339941, 'GPT-2': 8.367373396703011e-05, 'OPT': 5.730211141494202e-05, 'UniLM': 2.310083211467279e-05} 
 0.00400403288724895, 1
435


...

 combined: {'LLaMA': 0.7686715286598264, 'OPT': 0.09011142209565232, 'GPT-2': 0.07955270539438854, 'Human_write': 0.041587461641996934, 'T5': 0.007680168389671803, 'GPT-neo': 0.006783779110433404, 'UniLM': 0.0050707988052787235, 'Bloom': 0.0004335430461665624, 'BART': 0.00010859285658530157} 
 0.041587461641996934, 847
1468


100%|██████████| 1/1 [00:00<00:00, 17.14it/s]


 combined: {'GPT-neo': 0.4903978845093824, 'T5': 0.19703559043106667, 'BART': 0.17561258811181518, 'Bloom': 0.1354105967698767, 'Human_write': 0.0006211623010923913, 'UniLM': 0.0004162634852869403, 'LLaMA': 0.0002576183764148015, 'OPT': 0.00014277897482232576, 'GPT-2': 0.00010551704024259342} 
 0.0006211623010923913, 848
~~~ ai article ~~~
414


100%|██████████| 1/1 [00:00<00:00, 79.16it/s]

 combined: {'GPT-2': 0.521723072381702, 'LLaMA': 0.32961419702250805, 'OPT': 0.07747683348209483, 'UniLM': 0.026016346656075846, 'Human_write': 0.022684008563761376, 'T5': 0.01795036219846926, 'BART': 0.002560358332049835, 'GPT-neo': 0.0016990831284483272, 'Bloom': 0.0002757382348903425} 
 0.022684008563761376, 849


# Legacy Articles

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

In [ ]:
import numpy as np

score_nums = []

for score, _ in all_scores[0]:
  # print(score)
  score_nums.append(score)

mean = np.mean(score_nums)
median = np.median(score_nums)
std = np.std(score_nums)

print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Std dev: {std}")

# New Articles

In [ ]:
# input_file = "/content/scraped_wiki_articles_2_2025.json"
output_file = "/content/wikipedia_new.json"

process_files(input_file, output_file)
print(all_scores)
print(ai)
print(non_ai)

# Scraped Wiki Articles 8_2023

In [4]:
import json
import re

full_scores = []
all_scores = []

# Your llmdet function needs to be defined or imported here.

filenames = [
    # "CC-MAIN-2022-05.json",
    # "CC-MAIN-2023-40.json",
    # "CC-MAIN-2024-33.json",
    "CC-MAIN-2025-08.json",
]

def clean_text(text):
    text = text.replace('\n', '!@#').replace('\t', ' ')
    split_text = text.split('!@#')
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    text = ' '.join(split_text)
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)
    return text.strip()

def ai_score(lines, truncate_to=5000):
    non_ai, ai = 0, 0
    scores = []
    for ind, line in enumerate(lines):
        full_score = llmdet.detect(' '.join(line.strip().split()[:truncate_to]))[0]
        full_scores.append(full_score)
        score = full_score['Human_write']
        scores.append((score, ind))

        if max(full_score, key=full_score.get) != 'Human_write' and score < 4e-3:
            ai += 1
        else:
            non_ai += 1

    return scores, non_ai, ai

def process_files(file_names):
    for file_name in file_names:

        with open("/content/" + file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # texts = [clean_text(item["text"]) for item in data['train'] if len(item["text"].split()) >= 100]
        texts = [clean_text(item["text"]) for item in data['sites'] if len(item["text"].split()) >= 100]
        print(f"Processing {file_name}, number of texts: {len(texts)}")

        bino_scores, non_ai_count, ai_count = ai_score(texts)
        all_scores.append(bino_scores)

        print(f"{file_name}: Non-AI articles: {non_ai_count}, AI articles: {ai_count}")

process_files(filenames)


Processing CC-MAIN-2025-08.json, number of texts: 909


Streaming output truncated to the last 5000 lines.
100%|██████████| 1/1 [00:00<00:00,  2.63it/s]

CC-MAIN-2025-08.json: Non-AI articles: 570, AI articles: 339


# Common Crawl

wiki

Legacy: 249 AI 957 Human

New: 498 AI 1914 Human

8/2023: 405 AI 595 Human

8/2024: 309 AI 540 Human

---

crawl

2024-33: 346 AI 531 Human

2023-40: 340 AI 572 Human

2022-05: 382 AI 544 Human

2025-08: 339 AI 570 Human